# Бинарная классификация мошеннических транзакций
# Предварительный анализ данных

In [3]:
import pandas as pd

TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"

df = pd.read_csv(TRAIN_PATH)

print("Размер таблицы:", df.shape)  # (строки, столбцы)
display(df.head())  # первые 5 строк


Размер таблицы: (219129, 32)


,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,...,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0
1,1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,...,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0
2,2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,...,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0
3,3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,...,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0
4,4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,...,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0


In [4]:
df.info(memory_usage="deep")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219129 entries, 0 to 219128
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      219129 non-null  int64  
 1   Time    219129 non-null  float64
 2   V1      219129 non-null  float64
 3   V2      219129 non-null  float64
 4   V3      219129 non-null  float64
 5   V4      219129 non-null  float64
 6   V5      219129 non-null  float64
 7   V6      219129 non-null  float64
 8   V7      219129 non-null  float64
 9   V8      219129 non-null  float64
 10  V9      219129 non-null  float64
 11  V10     219129 non-null  float64
 12  V11     219129 non-null  float64
 13  V12     219129 non-null  float64
 14  V13     219129 non-null  float64
 15  V14     219129 non-null  float64
 16  V15     219129 non-null  float64
 17  V16     219129 non-null  float64
 18  V17     219129 non-null  float64
 19  V18     219129 non-null  float64
 20  V19     219129 non-null  float64
 21  V20     21

In [5]:
# проверка дубликатов строк
n_dupes = df.duplicated().sum()
print(f"Дубликаты: {n_dupes} ({n_dupes / len(df):.4%})")


Дубликаты: 0 (0.0000%)


In [7]:
# пропуски по столбцам
na_counts = df.isna().sum()
na_perc = na_counts / len(df) * 100
summary_na = (
    pd.DataFrame({"na_count": na_counts, "na_%": na_perc})
    .query("na_count > 0")
    .sort_values("na_count", ascending=False)
)

if summary_na.empty:
    print("✔️ Пропуски не обнаружены.")
else:
    display(summary_na)


✔️ Пропуски не обнаружены.


In [8]:
# распределение целевого признака
target_counts = df["Class"].value_counts().sort_index()
target_perc = target_counts / len(df) * 100
print("Распределение Class (0 — legit, 1 — fraud):")
display(pd.concat([target_counts, target_perc.rename("%")], axis=1))


Распределение Class (0 — legit, 1 — fraud):


,count,%
Class,,
0,218660,99.785971
1,469,0.214029


In [9]:
# базовые статистики по числовым полям
numeric_stats = df.describe().T  # .T = транспонировать для удобства
display(numeric_stats)


,count,mean,std,min,25%,50%,75%,max
id,219129.0,109564.000000,63257.237906,0.000000,54782.000000,109564.000000,164346.000000,219128.000000
Time,219129.0,62377.415376,25620.348569,0.000000,47933.000000,63189.000000,77519.000000,120580.000000
V1,219129.0,0.096008,1.395425,-29.807725,-0.846135,0.385913,1.190661,2.430494
V2,219129.0,0.048345,1.159805,-44.247914,-0.573728,0.046937,0.814145,16.068473
V3,219129.0,0.592102,1.132884,-19.722872,-0.027154,0.735895,1.306110,6.145578
V4,219129.0,0.069273,1.253125,-5.263650,-0.769256,0.064856,0.919353,12.547997
V5,219129.0,-0.161555,1.069530,-37.591259,-0.847346,-0.229929,0.356856,34.581260
V6,219129.0,0.133688,1.202411,-25.659750,-0.631835,-0.087778,0.482388,16.233967
V7,219129.0,-0.128224,0.817207,-31.179799,-0.646730,-0.098970,0.385567,39.824099
V8,219129.0,0.149534,0.716212,-28.903442,-0.095948,0.111219,0.390976,18.270586


In [10]:
# быстрый вывод ключевых фактов одной строкой (опционально)
print(f"""
👉 Строк: {len(df):,}
👉 Столбцов: {df.shape[1]}
👉 Пропуски: {df.isna().any().sum()} столбцов с NaN
👉 Дубликаты: {n_dupes}
👉 Fraud rate: {target_perc.loc[1]:.3f}%  (≈ 1 к {int(target_counts.loc[0] / target_counts.loc[1]):.0f})
""")



👉 Строк: 219,129
👉 Столбцов: 32
👉 Пропуски: 0 столбцов с NaN
👉 Дубликаты: 0
👉 Fraud rate: 0.214%  (≈ 1 к 466)



# Выводы

Пропусков нет → этап обработки пропусков упрощается: достаточно подтвердить это после трансформаций.

Все столбцы числовые. Категориальных признаков нет, поэтому кодирование (One-Hot, Ordinal и т. п.) не требуется.

Сильный дисбаланс классов (~0.2 %) — при моделировании понадобится:

- стратификация при сплите,

- метрики, устойчивые к дисбалансу (AUC-ROC, PR-AUC),

- техники балансировки (взвешивание классов, oversampling/undersampling) — опционально.

- id следует исключить из обучающих признаков (это индекс).

- Нормализация/стандартизация желательна: диапазоны Amount и PCA-компонент различаются; выберем метод (например, StandardScaler) на этапе подготовки данных.



# Пропуски в данных

In [16]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

print("✅ train shape:", df_train.shape)


✅ train shape: (219129, 32)


In [18]:
# функция проверки пропусков
def missing_report(df, name="dataframe"):
    """Возвращает таблицу пропусков (count, %) и печатает краткое резюме."""
    na_count = df.isna().sum()
    na_pct = na_count / len(df) * 100
    report = (
        pd.DataFrame({"na_count": na_count, "na_%": na_pct})
        .query("na_count > 0")
        .sort_values("na_count", ascending=False)
    )
    if report.empty:
        print(f"✔️  В '{name}' пропусков не обнаружено.")
    else:
        print(f"⚠️  Пропуски в '{name}':")
        display(report)
    return report


In [19]:
# отчёты по пропускам
missing_train = missing_report(df_train, "train")


✔️  В 'train' пропусков не обнаружено.


In [20]:
print(f"""
Итого:
• Столбцов с NaN в train: {len(missing_train)}
""")



Итого:
• Столбцов с NaN в train: 0



# Т.к. пропусков нет, можно продолжать работу с данными.

In [21]:
df_test = pd.read_csv(TEST_PATH)
print("✅ test.csv shape:", df_test.shape)
display(df_test.head())


✅ test.csv shape: (146087, 31)


,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,219129,120580.0,2.115519,-0.691809,-1.305514,-0.685655,-0.641265,-0.764784,-0.924262,-0.023030,...,0.067367,0.241708,0.682524,0.037769,-0.546859,-0.123055,-0.084889,0.004720,-0.021944,29.95
1,219130,120580.0,1.743525,-1.681429,-0.547387,-1.061113,-0.695825,2.458824,-1.632859,1.073529,...,0.441788,0.543278,1.294571,0.309541,3.703925,-0.242579,0.068708,0.002629,0.064690,163.50
2,219131,120580.0,2.205568,-1.571445,-0.238965,-1.662517,-1.652324,-0.054701,-1.682064,0.105613,...,-0.366906,-0.131527,0.086623,0.291375,0.739087,-0.543006,-0.297813,0.043699,-0.037855,16.00
3,219132,120580.0,1.989728,-0.972909,-1.938259,-1.440129,-0.166855,-0.794048,0.252889,-0.399789,...,-0.049136,-0.080115,-0.010732,-0.038550,0.656830,0.343470,-0.627529,-0.024338,-0.036143,120.98
4,219133,120580.0,-1.943548,-1.668761,0.363601,-0.977610,2.684779,-2.037681,0.039709,-0.048895,...,0.391627,0.083389,-0.306918,0.247822,-0.391799,-0.790716,-0.025706,0.330758,0.335537,1.98


In [22]:
#  функции для определения категориальных признаков
def find_categorical_columns(
        df: pd.DataFrame,
        max_unique_int: int = 20,
        max_unique_frac: float = 0.05,
) -> list[str]:
    """
    Возвращает список столбцов, которые можно считать категориальными.

    • object / category dtype — всегда.
    • int/float dtype — если количество уникальных
      1) ≤ max_unique_int И
      2) доля уникальных ≤ max_unique_frac (по умолчанию ≤5 %).
    """
    cat_cols = []

    for col in df.columns:
        dtype = df[col].dtype

        # 1. Явно категориальные типы
        if dtype == "object" or str(dtype).startswith("category"):
            cat_cols.append(col)
            continue

        # 2. Числовые с ограниченным числом уникальных значений
        if pd.api.types.is_numeric_dtype(dtype):
            uniq = df[col].nunique(dropna=False)
            uniq_frac = uniq / len(df)
            if (uniq <= max_unique_int) and (uniq_frac <= max_unique_frac):
                cat_cols.append(col)

    return cat_cols


In [23]:
# поиск и отчёт
categorical_cols = find_categorical_columns(df_test)

if categorical_cols:
    print("🗂️ Найдены потенциально категориальные признаки:")
    for c in categorical_cols:
        uniq = df_test[c].nunique(dropna=False)
        print(f"• {c:<15}  (уникальных: {uniq})")
else:
    print("✔️  Категориальные признаки не обнаружены.")


✔️  Категориальные признаки не обнаружены.


In [24]:
# сводка по уникальным значениям
# Показывает топ-10 признаков с наименьшим числом уникальных значений
(
    df_test.apply(lambda s: s.nunique(dropna=False))
    .sort_values()
    .head(10)
    .to_frame("unique_values")
    .style.format(thousands=" ")
)


,unique_values
Amount,15 594
Time,23 662
V3,145 117
V6,145 136
V5,145 154
V4,145 164
V11,145 173
V9,145 188
V14,145 191
V1,145 194


# Нормирование данных
Метод нормирования: StandardScaler (Z-score: (x − μ)/σ )

- Центрирует и приводит всё к дисперсии 1.
- Устойчив для признаков, уже похожих на нормальное распределение (PCA-компоненты V1–V28).


In [33]:
#  выделяем признаки и целевую переменную
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib  # чтобы сохранить scaler на будущее

TARGET_COL = "Class"
ID_COL = "id"

X_train = df_train.drop(columns=[TARGET_COL, ID_COL])
y_train = df_train[TARGET_COL]

X_test = df_test.drop(columns=[ID_COL])  # в test нет Class




In [29]:
# создаём пайплайн со StandardScaler
num_features = X_train.columns.tolist()  # все оставшиеся числовые

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features)
    ],
    remainder="drop"
)

pipeline = Pipeline([
    ("scaler", preprocess)
])


In [34]:
# 🟢 Ячейка 5 — фит и трансформ
X_train_scaled = pipeline.fit_transform(X_train)
X_test_scaled = pipeline.transform(X_test)

print("Масштабированные формы:", X_train_scaled.shape, X_test_scaled.shape)


Масштабированные формы: (219129, 30) (146087, 30)


In [35]:
# преобразуем обратно в DataFrame
scaled_cols = num_features
df_train_scaled = pd.DataFrame(X_train_scaled, columns=scaled_cols)
df_train_scaled[TARGET_COL] = y_train.values  # возвращаем цель

df_test_scaled = pd.DataFrame(X_test_scaled, columns=scaled_cols)
df_train_scaled.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-2.434688,1.417723,-0.153277,-1.526656,0.274174,-0.029063,-1.117617,0.292364,-0.576662,0.749882,...,-0.718199,-1.400090,1.217046,-0.181829,-1.021451,0.417528,-0.388293,-0.466403,-0.426937,0
1,-2.434688,1.363615,-1.120222,-0.982513,-0.769129,-0.898502,-0.336453,-1.102617,-0.138696,-0.058446,...,0.203208,0.020885,0.578455,-0.773794,-1.449321,-1.001415,-0.100477,-0.336961,0.116982,0
2,-2.434688,-0.003205,0.824427,-0.719888,-0.402612,0.775205,-0.933160,1.317136,-0.327171,0.005986,...,-0.699333,-1.259402,0.644384,1.608883,-1.551175,0.158620,0.901735,0.424868,-0.422228,0
3,-2.434688,1.349872,-0.201150,-1.462031,0.040580,-0.050193,-0.650792,0.049435,-0.258165,0.886196,...,-0.152446,-0.048410,0.685889,-0.067352,-0.302912,-0.223754,-0.307748,-0.552913,-0.433436,0
4,-2.434688,0.666386,-0.189836,0.539876,0.937360,-0.444137,0.802877,-0.991706,0.586008,0.702944,...,0.308014,1.103629,0.246506,-0.438073,0.329029,-0.554398,0.404124,0.022671,-0.433436,0


In [37]:
# сохраняем scaler и/или готовые таблицы
joblib.dump(pipeline, "standard_scaler_pipeline.joblib")
df_train_scaled.to_csv("train_scaled.csv", index=False)
df_test_scaled.to_csv("test_scaled.csv", index=False)


# Выбросы

In [42]:
from scipy import stats
import numpy as np


num_cols = df.select_dtypes(include=[np.number]).columns.tolist()



In [40]:
# расчёт z-score и маски выбросов
z_scores = np.abs(stats.zscore(df[num_cols], axis=0, nan_policy="omit"))
outlier_mask = z_scores > 3  # True, где |z| > 3


In [41]:
# сводная таблица по выбросам
summary = (
    pd.DataFrame({
        "feature": num_cols,
        "outliers": outlier_mask.sum(axis=0),
    })
    .assign(
        outlier_pct=lambda d: d["outliers"] / len(df) * 100,
        min_val=lambda d: [df[col].min() for col in d["feature"]],
        max_val=lambda d: [df[col].max() for col in d["feature"]],
        method="3-σ"
    )
    .loc[lambda d: d["outliers"] > 0]  # оставляем только признаки с выбросами
    .sort_values("outliers", ascending=False)
    .reset_index(drop=True)
)

display(summary)  # красивый вывод в Jupyter


,feature,outliers,outlier_pct,min_val,max_val,method
0,V6,4347,1.983763,-25.659750,16.233967,3-σ
1,Amount,3695,1.686221,0.000000,7475.000000,3-σ
2,V27,3526,1.609098,-9.234767,13.123618,3-σ
3,V20,3404,1.553423,-18.679066,15.407839,3-σ
4,V23,3061,1.396894,-11.958588,12.734391,3-σ
5,V8,2907,1.326616,-28.903442,18.270586,3-σ
6,V28,2785,1.270941,-4.551680,23.263746,3-σ
7,V2,2686,1.225762,-44.247914,16.068473,3-σ
8,V21,2523,1.151377,-14.689621,22.062945,3-σ
9,V14,2407,1.098440,-15.585021,6.078453,3-σ


Всего 16 из 30 числовых признаков имеют ≥ 1 % наблюдений за пределами 3 стандартных отклонений; остальные находятся в диапазоне 0.3–0.9 %.

Сводно:

29 270 транзакций (13.36 % выборки) содержат хотя бы один «сигма-выброс».

Наиболее «тяжёлый хвост» — у PCA-компонент V6, V27, V20, V23.

Amount выражено в евро; несмотря на долгий хвост (до €7 475), по 3-σ-критерию выбросами считается лишь 1.7 %.

Однако по правилу IQR*1.5 все суммы > €163 (или < 0 €) дают 20 617 выбросов (9.4 %).

# Варианты нормализации

| Вариант                                    | Преимущества                                                                                                   | Недостатки                                                              |
| ------------------------------------------ | -------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------- |
| **RobustScaler** (медиана / IQR)           | Снижает влияние длинных хвостов `Amount`, `V6` и др.                                                           | Лёгкое искажение признаков, где распределение уже близко к нормальному. |
| **Клиппинг** (Winsorize ±3 σ или IQR\*1.5) | Убирает экстремальные значения без удаления строк.                                                             | Потеря информации; порог выбирается эвристически.                       |
| **Оставить как есть**                      | PCA-признаки часто ведут себя как норм. распределение с редкими «краями»; модели на основе деревьев устойчивы. | Линейные модели чувствительны; может ухудшить масштабирование.          |

